In [1]:
pip install tensorflow deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.2 MB/s eta 0:00:00


In [2]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from deap import base, creator, tools, algorithms

In [3]:
# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [4]:
# Define the fitness function and individual
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

In [5]:
def create_individual():
    """Create an individual with random hyperparameters."""
    # Ensure dropout rate is always in range (0,1)
    use_dropout = random.choice([True, False])
    dropout_rate = random.uniform(0.2, 0.5) if use_dropout else 0

    return creator.Individual([
        random.randint(1, 3),  # Number of hidden layers
        random.randint(32, 512),  # Number of units in layer 1
        use_dropout,  # True if dropout should be used
        dropout_rate  # Dropout rate
    ])

In [6]:
def evaluate(individual):
    """Evaluate the performance of an individual."""
    num_layers, units_1, use_dropout, dropout_rate = individual

    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))

    for _ in range(num_layers):
      model.add(Dense(units_1, activation='relu'))
      if use_dropout:
        # Only add Dropout layer if use_dropout is True
        model.add(Dropout(dropout_rate))

    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=3, batch_size=128, verbose=0, validation_split=0.2)

    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    return accuracy,

In [7]:
# Setup the toolbox for DEAP
toolbox = base.Toolbox()
toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

In [8]:
# Run the evolutionary algorithm
population = toolbox.population(n=10)
ngen = 5  # Number of generations
cxpb = 0.5  # Crossover probability
mutpb = 0.2  # Mutation probability

In [ ]:
algorithms.eaSimple(population, toolbox, cxpb=cxpb, mutpb=mutpb, ngen=ngen, stats=None, halloffame=None, verbose=True)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


gen	nevals
0  	10    


In [ ]:
# Find and print the best individual
best_individual = tools.selBest(population, 1)[0]
print(f'Best individual: {best_individual}, Accuracy: {evaluate(best_individual)[0]}')